In [25]:
import sys
sys.path.append('../model/dataset.py')
sys.path.append('..')
from model import dataset
from utils import import_data
images_with_ball, images_without_ball = import_data('../predictions/image10.jpg')
train_dataset = dataset.ImageDataset(
    images_with_ball,
    images_without_ball,
    transforms=None
)

In [36]:
import numpy as np
import torch
print(len(train_dataset))
image, label = train_dataset[1100]
filter = torch.nn.Conv2d(3, 3, kernel_size=3, bias=False, stride=1)
image = filter(image)
image = image.permute(1 , 2 , 0).detach().numpy()
print(image.shape)
image = image.astype(np.uint8)
print(label)
import cv2
cv2.imshow('img', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1152
(58, 58, 3)
tensor(1.)


6928.0


KeyboardInterrupt: 